In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"
cohort = "GSE145037"

# Input paths
in_trait_dir = "../../input/GEO/Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Rectal_Cancer/GSE145037"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/GSE145037.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/GSE145037.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/GSE145037.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Hypermethylation and downregulation of UTP6 are associated with stemness properties, chemoradiotherapy resistance and prognosis in rectal cancer: A co-expression network analysis"
!Series_summary	"To measure global gene expression in primary locally advanced rectal cancer patients who have undergone CRT and screen valuable biomarkers to predict the effects of CRT.Samples fromprimary locally advanced rectal cancer patients were collected. The effects of chemoradiotherapy were evaluated."
!Series_overall_design	"All patients underwent standard CRT  after signing the chemoradiotherapy agreement; subsequently, they were evaluated in accordance with the AJCC tumor regression grade (TRG).Each samplewas collected before CRT. Each sample was stored in liquid nitrogen until total RNA extraction."
Sample Characteristics Dictionary:
{0: ['tissue: primary rectalcancer'], 1: ['Sex: Male', 'Sex: Female'], 2: ['age: 34', 'age: 66', 'age: 69', 'age: 65', 'age: 72

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and design mentioning "gene expression", this appears to be gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait - response to CRT is available in row 3
trait_row = 3

# For gender - available in row 1
gender_row = 1

# For age - available in row 2
age_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert treatment response to binary: 1 for response, 0 for non-response"""
    if not isinstance(value, str):
        return None
    value = value.lower()
    if 'response to the crt:' in value:
        value = value.split('response to the crt:')[1].strip()
        if 'response' == value:
            return 1
        elif 'non-response' == value:
            return 0
    return None

def convert_age(value):
    """Convert age to a continuous value"""
    if not isinstance(value, str):
        return None
    if 'age:' in value:
        try:
            age = int(value.split('age:')[1].strip())
            return age
        except:
            pass
    return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male"""
    if not isinstance(value, str):
        return None
    value = value.lower()
    if 'sex:' in value:
        value = value.split('sex:')[1].strip().lower()
        if 'female' in value:
            return 0
        elif 'male' in value:
            return 1
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, 
                             info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The dictionary format suggests rows are indexed by integers (0-5)
    # and each row contains a list of values
    sample_char_dict = {
        0: ['tissue: primary rectalcancer'],
        1: ['Sex: Male', 'Sex: Female'],
        2: ['age: 34', 'age: 66', 'age: 69', 'age: 65', 'age: 72', 'age: 64', 'age: 53', 'age: 60', 'age: 44', 
            'age: 58', 'age: 41', 'age: 52', 'age: 48', 'age: 49', 'age: 61', 'age: 63', 'age: 75', 'age: 46', 
            'age: 59', 'age: 70', 'age: 68', 'age: 73'],
        3: ['response to the crt: non-response', 'response to the crt: response'],
        4: ['clincal t stage: 4', 'clincal t stage: 3', 'clincal t stage: 2'],
        5: ['clincal n positive: 1', 'clincal n positive: 0']
    }
    
    # Convert to DataFrame format suitable for geo_select_clinical_features
    clinical_data = pd.DataFrame.from_dict(sample_char_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0, 34.0, 1.0], 1: [1.0, 66.0, 0.0], 2: [nan, 69.0, nan], 3: [nan, 65.0, nan], 4: [nan, 72.0, nan], 5: [nan, 64.0, nan], 6: [nan, 53.0, nan], 7: [nan, 60.0, nan], 8: [nan, 44.0, nan], 9: [nan, 58.0, nan], 10: [nan, 41.0, nan], 11: [nan, 52.0, nan], 12: [nan, 48.0, nan], 13: [nan, 49.0, nan], 14: [nan, 61.0, nan], 15: [nan, 63.0, nan], 16: [nan, 75.0, nan], 17: [nan, 46.0, nan], 18: [nan, 59.0, nan], 19: [nan, 70.0, nan], 20: [nan, 68.0, nan], 21: [nan, 73.0, nan]}
Clinical data saved to ../../output/preprocess/Rectal_Cancer/clinical_data/GSE145037.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS',
       'AACS', 'AADAC', 'AADACL2', 'AADACL3', 'AADACL4', 'AADAT', 'AAGAB',
       'AAK1', 'AAMDC', 'AAMP', 'AANAT', 'AAR2'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers shown in the output are human gene symbols.
# These are standard HUGO Gene Nomenclature Committee (HGNC) gene symbols,
# including known genes like A1BG (alpha-1-B glycoprotein), A2M (alpha-2-macroglobulin),
# and AANAT (aralkylamine N-acetyltransferase).
# No mapping is required as these are already in the proper gene symbol format.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Load the previously saved clinical data instead of re-extracting
clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data shape: {clinical_features.shape}")

# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# Inspect the data for missing values before handling them
print(f"Missing values in trait column: {linked_data[trait].isna().sum()}/{len(linked_data)}")
print(f"Missing values in Age column: {linked_data['Age'].isna().sum()}/{len(linked_data)}")
print(f"Missing values in Gender column: {linked_data['Gender'].isna().sum()}/{len(linked_data)}")

# 3. Handle missing values in the linked data
linked_data_processed = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data_processed.shape}")

# Add validation check - if no samples remain, note the issue
if linked_data_processed.shape[0] == 0:
    print("No samples remain after handling missing values. The dataset cannot be processed further.")
    is_trait_biased = True  # Mark as biased since we can't use it
    unbiased_linked_data = linked_data_processed
else:
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_processed, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from rectal cancer patients, focusing on response to chemoradiotherapy. However, high levels of missing trait values make it unsuitable for association studies."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Loaded clinical data shape: (3, 22)
Normalized gene data shape: (18404, 31)
First few normalized gene symbols: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC']


Normalized gene data saved to ../../output/preprocess/Rectal_Cancer/gene_data/GSE145037.csv
Linked data shape: (53, 18407)
   Rectal_Cancer   Age  Gender  A1BG  A1CF  A2M  A2ML1  A3GALT2  A4GALT  \
0            0.0  34.0     1.0   NaN   NaN  NaN    NaN      NaN     NaN   
1            1.0  66.0     0.0   NaN   NaN  NaN    NaN      NaN     NaN   
2            NaN  69.0     NaN   NaN   NaN  NaN    NaN      NaN     NaN   
3            NaN  65.0     NaN   NaN   NaN  NaN    NaN      NaN     NaN   
4            NaN  72.0     NaN   NaN   NaN  NaN    NaN      NaN     NaN   

   A4GNT  ...  ZW10  ZWILCH  ZWINT  ZXDA  ZXDB  ZXDC  ZYG11A  ZYG11B  ZYX  \
0    NaN  ...   NaN     NaN    NaN   NaN   NaN   NaN     NaN     NaN  NaN   
1    NaN  ...   NaN     NaN    NaN   NaN   NaN   NaN     NaN     NaN  NaN   
2    NaN  ...   NaN     NaN    NaN   NaN   NaN   NaN     NaN     NaN  NaN   
3    NaN  ...   NaN     NaN    NaN   NaN   NaN   NaN     NaN     NaN  NaN   
4    NaN  ...   NaN     NaN    NaN   NaN 